In [5]:
%pip install langgraph langchain_openai langfuse

  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
Using cached packaging-24.2-py3-none-any.whl (65 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10/10 [langfuse]/10 [langfuse]ter-otlp-proto-grpc]
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

from typing import TypedDict, List, Dict, Any, Optional
from langgraph.graph import StateGraph, START, END
from smolagents import LiteLLMModel
from langchain.schema import BaseMessage, ChatMessage
from langchain_core.messages import HumanMessage, AIMessage

In [17]:
class EmailState(TypedDict):
    email: Dict[str, Any]
    is_spam: Optional[bool]
    spam_reason: Optional[str]
    email_category: Optional[str]
    email_draft: Optional[str]
    messages: List[Dict[str, Any]]

from langfuse.langchain import CallbackHandler
lf_handler = CallbackHandler()

model = LiteLLMModel(
    model_id="gemini/gemini-2.0-flash-lite",
    temperature=0.0,
    max_tokens=1024,
    callbacks=[lf_handler]
)

Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 


In [18]:
def read_email(state: EmailState):
    """Agent reads and logs the incoming email"""
    email = state['email']
    # Here we might do some initial preprocessing
    print(f"Alfred is processing an email from {email['sender']} with subject: {email['subject']}")

    return{}

def classify_email(state: EmailState):
    """Agent uses an LLM to determine whether an email is spam or legitamate"""
    email = state['email']

    prompt = f"""
    As my agent, analyze this email and determine if it is spam or legitimate.

    Analyse the e-mail below.
    1. Say whether it is spam.
    2. If spam, explain why. 
    3. If not spam, label it with a broad category (inquiry, complaint, thank-you, request, info).
    
    Email:
    From: {email['sender']}
    Subject: {email['subject']}
    Body: {email['body']}
    """

    messages=[HumanMessage(content=prompt)]
    response = model.generate([messages])
    response_text = response.content.lower()
    
    is_spam="spam" in response_text and "not spam" not in resresponse_textponsetext

    spam_reason = None
    if is_spam and "reason: " in response_text:
        spam_reason = response_text.split("reason:")[1].strip()

    email_cat = None
    if not is_spam:
        categories = ["inquiry", "complaint", "thank you", "request", "information"]
        for category in categories:
            if category in response_text:
                email_category = category
                break
    
    new_message = state.get("messages", []) + [
        {"role":"user", "content":prompt},
        {"role":"assisstant", "content":response.content}
    ]

    return {
        "is_spam": is_spam,
        "spam_reason": spam_reason,
        "email_category": email_category,
        "messages": new_message,
    }

In [20]:
def handle_spam(state: EmailState):
    """Agent discards spam email with a note"""
    print(f"Agent has marked the email as spam. Reason: {state['spam_reason']}")
    print("The email has been moved to the spam folder.")
    
    return {}

def draft_response(state: EmailState):
    """Agent drafts a preliminary response for legitimate emails"""
    email = state["email"]
    category = state["email_category"] or "general"

    prompt = f"""
    As my Agent, draft a polite preliminary response to this email.
    
    Email:
    From: {email['sender']}
    Subject: {email['subject']}
    Body: {email['body']}
    
    This email has been categorized as: {category}
    
    Draft a brief, professional response that I can review and personalize before sending.
    """

    messages = [HumanMessage(content = prompt)]
    response = model.generate(messages)

    new_msgs = state["messages"] + [
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": response.content},
    ]
    return {"email_draft": response.content, "messages": new_msgs}

def notify_me(state: EmailState):
    """
    Notify me when an email comes in along with its classification
    """
    e = state["email"]
    print(f"\nSir, new mail from {e['sender']} ({state['email_category']})")
    print(state["email_draft"])
    return {}

def route_email(state: EmailState) -> str:
    return "spam" if state["is_spam"] else "legitimate" 

In [ ]:
g = StateGraph(EmailState)

g.add_node("read_email", read_email)
g.add_node("classify_email", classify_email)
g.add_node("handle_spam", handle_spam)
g.add_node("draft_response", draft_response)
g.add_node("notify_mr_wayne", notify_me)

